In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.2 MB/s eta 0:00:00


In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=2108f8e89c3d7d8a2e3bba8d46d672b6e4d446af7c7389bd8bac610719a45375
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 13.1

In [5]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [6]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [7]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [8]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [9]:
filter_phrase = df['tags'] == "passage"
df_filtered = df[filter_phrase]
# print (df_filtered)

In [10]:
filter_phrase_validation = df_validation['tags'] == "passage"
df_filtered_validation = df_validation[filter_phrase_validation]
# print (df_filtered_validation)

In [11]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [12]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [ ]:
# wordLemm = WordNetLemmatizer()
# def processed_paragraph(paragraph):
#     # Lower Casing
#     paragraph = paragraph.lower()
#     # Remove punctuations
#     paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
#     # Tokenizing words
#     tokens = word_tokenize(paragraph)
#     # Removing Stop Words
#     final_tokens = [w for w in tokens if w not in stopword]
#     # Reducing a word to its word stem using lemmatization
#     finalwords=[]
#     for w in final_tokens:
#       if len(w)>1:
#         word = wordLemm.lemmatize(w)
#         finalwords.append(word)
#     return ' '.join(finalwords)

In [13]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [14]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
new_df = generate_new_df(df_filtered)

In [17]:
new_df_validation = generate_new_df(df_filtered_validation)

In [18]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [19]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [20]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# questions= new_df['postText']
# answers= new_df['spoiler']
# stories_not_summarized = new_df['paragraph']
# tags = new_df['tags']

# print(stories_not_summarized[0])




In [21]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ac20f804060cbd794341d114350b5a3c9e7c0315b558ce69fff6d7c0be40a4d1
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [22]:
import torch

In [ ]:
# from transformers import pipeline
# import requests
# import pprint
# import time
# pp = pprint.PrettyPrinter(indent=14)

# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')


# # model = T5ForConditionalGeneration.from_pretrained('t5-small')
# # tokenizer = T5Tokenizer.from_pretrained('t5-small')
# device_cuda = torch.device('cuda')

In [23]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [24]:
from datetime import datetime

In [25]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

wandb: Currently logged in as: pradhaneva94 (evapradhan). Use `wandb login --relogin` to force relogin


In [ ]:
# # summarize with T5
# # summarizer_t5 = pipeline(task='summarization', model="t5-large")
# summarizer_t5 = pipeline(task='summarization', model="t5-small")

In [ ]:
#FINE TUNING START

In [26]:
!pip install transformers #==2.9.0
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.2 MB/s eta 0:00:00


In [27]:
from datasets import Dataset
from datasets import dataset_dict

In [28]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
import torch
import numpy as np
import pandas as pd
from transformers import (TrainingArguments, Trainer, EarlyStoppingCallback)
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [31]:
# max_input = 512
# max_target = 128
# batch_size = 3
model_checkpoints = "t5-small"

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [36]:
input_text = "Sumarize:" + "condition: " + new_df['postText'] + " paragraph: " + new_df["paragraph"]
target_text = new_df["spoiler"]


data_to_process_train = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [37]:
data_to_process_train

,input_ids,labels
0,Sumarize:condition: wes welker wanted dinner w...,how about that morning we go throw?
5,Sumarize:condition: what happens if your new a...,"Apple says that if AirPods are lost or stolen,..."
6,Sumarize:condition: the reason why gabor kiral...,"""The more good games I had in them, the more I..."
7,Sumarize:condition: you ’ ll never believe wha...,rainbow colours in the sky and a halo spanning...
8,Sumarize:condition: should you drink red wine ...,Red wine is clearly the drink of choice if you...
...,...,...
3189,Sumarize:condition: women reveal what they thi...,almost every other part of a man is considered...
3193,Sumarize:condition: heres another huge reason ...,may cut your risk for colorectal cancer -- the...
3194,Sumarize:condition: if you see a purple butter...,one of her unborn twins had a condition called...
3195,Sumarize:condition: has facebooks video explos...,it hasn’t necessarily taken the wind out of Yo...


In [38]:
df1_dict = data_to_process_train.to_dict(orient = 'list')

In [39]:
dict_train_ = Dataset.from_dict(df1_dict)

In [40]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [41]:
input_text = "Sumarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_validation = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [42]:
data_to_process_train

,input_ids,labels
0,Sumarize:condition: wes welker wanted dinner w...,how about that morning we go throw?
5,Sumarize:condition: what happens if your new a...,"Apple says that if AirPods are lost or stolen,..."
6,Sumarize:condition: the reason why gabor kiral...,"""The more good games I had in them, the more I..."
7,Sumarize:condition: you ’ ll never believe wha...,rainbow colours in the sky and a halo spanning...
8,Sumarize:condition: should you drink red wine ...,Red wine is clearly the drink of choice if you...
...,...,...
3189,Sumarize:condition: women reveal what they thi...,almost every other part of a man is considered...
3193,Sumarize:condition: heres another huge reason ...,may cut your risk for colorectal cancer -- the...
3194,Sumarize:condition: if you see a purple butter...,one of her unborn twins had a condition called...
3195,Sumarize:condition: has facebooks video explos...,it hasn’t necessarily taken the wind out of Yo...


In [43]:
df2_dict = data_to_process_validation.to_dict(orient = 'list')

In [44]:
dict_validation_ = Dataset.from_dict(df2_dict)

In [45]:
dict_validation_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 322
})

In [46]:
input_text = "Sumarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_test = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [47]:
data_to_process_test

,input_ids,labels
0,Sumarize:condition: five nights at freddy ’ s ...,some of the plot elements are so disturbing th...
4,Sumarize:condition: a man swallowed a microsd ...,a man who swallowed a 64GB microSD card and th...
7,Sumarize:condition: you wont believe this stun...,McGonagall was appointed as Dumbledore’s assis...
8,Sumarize:condition: jj abrams has an answer on...,All the scenes are actually in the movie
9,Sumarize:condition: kristin cavallari kristinc...,"""I had fake relationships, fake fights. I don'..."
...,...,...
792,Sumarize:condition: one guy decided to explore...,"This was clearly a crack house, and he was sta..."
793,Sumarize:condition: heres a controversial idea...,China is too quick to rebalance its services s...
794,Sumarize:condition: dog dies one hour after hi...,the plant the dog was chewing on was deadly wa...
795,Sumarize:condition: this is what happens when ...,The video below shows the stunned cleaner init...


In [48]:
df3_dict = data_to_process_test.to_dict(orient = 'list')

In [49]:
dict_test_ = Dataset.from_dict(df3_dict)

In [50]:
dict_test_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 322
})

In [ ]:
# df1_dict = data_to_process_train.to_dict(orient = 'list')
# df2_dict = data_to_process_validation.to_dict(orient = 'list')
# df3_dict = data_to_process_test.to_dict(orient = 'list')


# dict_train_ = Dataset.from_dict(df1_dict)
# dict_validation_ = Dataset.from_dict(df2_dict)
# dict_test_ = Dataset.from_dict(df3_dict)

In [51]:
dict_all_2 = dataset_dict.DatasetDict({"train":dict_train_, "validation":dict_validation_, "test": dict_test_})

In [52]:
dict_all_2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1274
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 322
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 322
    })
})

In [53]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [54]:
model = "t5-small"

In [55]:
tokenizer1 = tokenizer.from_pretrained("t5-small")

In [56]:
def preprocess_data(example):
    input_text = example['input_ids']
    target_text = example['labels']

    input_ids = tokenizer1(input_text, padding="max_length", truncation=True, max_length= 1000).input_ids

    with tokenizer1.as_target_tokenizer():
      target_ids = tokenizer1(target_text, padding='max_length', truncation=True, max_length= 500).input_ids
    # target_ids = tokenizer(target_text, padding="max_length", truncation=True, max_length= 500, return_tensors="pt").input_ids
    
    return {"input_ids": input_ids, "labels": target_ids}
    # return {"input_text": input_text, "target_text": target_text}

In [57]:
train_dataset = dict_train_
test_dataset = dict_validation_

train_dataset_processed = train_dataset.map(preprocess_data)
test_dataset_processed = test_dataset.map(preprocess_data)

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

In [60]:
train_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [61]:
test_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 322
})

In [62]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [63]:
model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, num_labels = 4)

In [64]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.9 MB/s eta 0:00:00


In [65]:
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
bertscore_metric = load_metric("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [66]:
#out_1 = []
#out_2 = []

def compute_metrics(eval_pred):
    logits_full, labels = eval_pred

    #out_1.append(logits_full[0])
    #out_2.append(logits_full[1])

    predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
    decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds_bleu = [pred.split() for pred in decoded_preds]
    decoded_labels_bleu = [[pred.split()] for pred in decoded_labels]
    
    bleu_score = bleu_metric.compute(predictions=decoded_preds_bleu, references= decoded_labels_bleu)
    meteor_score = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bertscore_score = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    
    return {"bleu": bleu_score["bleu"], "meteor": meteor_score["meteor"], "bertscore": np.mean(bertscore_score["f1"])}

In [67]:
import shutil


In [68]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/T5_finetuning__passage_results_conditional_all_sizelimited",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics
)


In [69]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.169600,0.141147
2,0.131200,0.116559
3,0.103600,0.101116
4,0.093500,0.090724
5,0.090800,0.079301
6,0.075000,0.073602
7,0.092000,0.070278
8,0.078200,0.066624
9,0.078200,0.064068
10,0.068400,0.063737


TrainOutput(global_step=7975, training_loss=0.10223105660800276, metrics={'train_runtime': 2921.8506, 'train_samples_per_second': 10.901, 'train_steps_per_second': 2.729, 'total_flos': 8419211673600000.0, 'train_loss': 0.10223105660800276, 'epoch': 25.0})

In [70]:
metrics = trainer.evaluate()


In [71]:
metrics

{'eval_loss': 0.05783680081367493,
 'eval_runtime': 9.6498,
 'eval_samples_per_second': 33.369,
 'eval_steps_per_second': 8.394,
 'epoch': 25.0}

In [ ]:
dict_validation_["labels"]

In [72]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["labels"],
        padding="max_length",
        truncation=True,
        max_length=1000,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [73]:
# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summaries_after_tuning_all = []

def generate_batch(lst, batch_size):
    """  Yields batch of specified size """
    for i in range(0, len(lst), batch_size):
        yield lst[i : i + batch_size]
        # summaries_after_tuning = generate_summary(test_samples, model)[1]
        # summaries_after_tuning_all.append(summaries_after_tuning)


# test_samples = dict_validation_.select(range())
test_samples = generate_batch(dict_validation_, 16)

# # summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]

for batch in test_samples:
  # print(batch)
  summaries_after_tuning = generate_summary(batch, model)[1]
  summaries_after_tuning_all.append(summaries_after_tuning)

In [74]:
summaries_after_tuning_all

[['Einige der plot elements are so disturbing that they make him feel sick',
  'Ein Mann, der swallowed a 64GB microSD card and then pooped',
  'McGonagall was appointed as Dumbledore’s assistant in 1956, not',
  'Alle Szenen sind eigentlich in dem Film.',
  '"I had fake relationships, fake fights. I don\'t care anymore, I can',
  "Er'd eaten a peanut butter sandwich and wasn't aware of her peanut allergy.",
  "'She kicked her and got into a fight with her current boyfriend.",
  'Dunham picked the boy up and took him to a Subway to get something to ',
  'Aubrey has cerebral palsy, but she was neglected and abused by her biological',
  "Leider, there's not enough solid research out there on whether melatonin",
  "Kevin Williamson said he didn't want to write it.",
  'Sophie and Riley were considered "micro-preemies" and suffered a s',
  'Buss and Jackson announced that they were mutually ending their four-year engagement.',
  'In the video, an abstract, wooden sculpture clock appeared n

'Einige der plot elements are so disturbing that they make him feel sick.'

In [75]:
stories_list =[]
for i in range(len(summaries_after_tuning_all)):
  for j in range(len(summaries_after_tuning_all[i])):
    summary=summaries_after_tuning_all[i][j]
    stories_list.append(summary)


print(stories_list)

['Einige der plot elements are so disturbing that they make him feel sick', 'Ein Mann, der swallowed a 64GB microSD card and then pooped', 'McGonagall was appointed as Dumbledore’s assistant in 1956, not', 'Alle Szenen sind eigentlich in dem Film.', '"I had fake relationships, fake fights. I don\'t care anymore, I can', "Er'd eaten a peanut butter sandwich and wasn't aware of her peanut allergy.", "'She kicked her and got into a fight with her current boyfriend.", 'Dunham picked the boy up and took him to a Subway to get something to ', 'Aubrey has cerebral palsy, but she was neglected and abused by her biological', "Leider, there's not enough solid research out there on whether melatonin", "Kevin Williamson said he didn't want to write it.", 'Sophie and Riley were considered "micro-preemies" and suffered a s', 'Buss and Jackson announced that they were mutually ending their four-year engagement.', 'In the video, an abstract, wooden sculpture clock appeared not to be working, so a', 'B

In [ ]:
# type(new_df['spoiler'][0:40])

In [ ]:
# type(stories_list[0:40])

list

In [ ]:
# stories_list_series = pd.Series((v for v in stories_list))

In [76]:
import pandas as pd

In [77]:
dataframe = pd.DataFrame({ 'postText': new_df_validation["postText"], 'actual_spoiler':test_dataset['labels'], 'gen_spoiler':stories_list })

In [78]:
dataframe

,postText,actual_spoiler,gen_spoiler
0,five nights at freddy ’ s sequel delayed for w...,some of the plot elements are so disturbing th...,Einige der plot elements are so disturbing tha...
4,a man swallowed a microsd card and you wont be...,a man who swallowed a 64GB microSD card and th...,"Ein Mann, der swallowed a 64GB microSD card an..."
7,you wont believe this stunning harry potter re...,McGonagall was appointed as Dumbledore’s assis...,McGonagall was appointed as Dumbledore’s assis...
8,jj abrams has an answer on if there will be a ...,All the scenes are actually in the movie,Alle Szenen sind eigentlich in dem Film.
9,kristin cavallari kristincav opens up about ju...,"""I had fake relationships, fake fights. I don'...","""I had fake relationships, fake fights. I don'..."
...,...,...,...
792,one guy decided to explore this abandoned buil...,"This was clearly a crack house, and he was sta...",a dealer stash. People have gotten killed for ...
793,heres a controversial idea about chinas econom...,China is too quick to rebalance its services s...,China is too quick to re-equilibrieren its ser...
794,dog dies one hour after hiking with his owner ...,the plant the dog was chewing on was deadly wa...,"Die plant, auf die die Hund Hund chewed on, wa..."
795,this is what happens when you leave a hotel cl...,The video below shows the stunned cleaner init...,The video below shows the stunned cleaner init...


In [96]:
file = open("T5_conditional_finetuning_no_tags_passage_results.txt", "w")

#convert variable to string
# str_1 = repr(dataframe)
str2 = dataframe.to_json()
file.write("T5_conditional_finetuning_no_tags_passage_results = " + str2 + "\n")

#close file
file.close()

In [80]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄█
eval/samples_per_second,█▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▁
eval/steps_per_second,█▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
#EVALUATION METRICS

In [81]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [82]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [83]:
bleu_scores

[0.7672486789360017,
 0.6555164869596242,
 0.7545475154775315,
 0.14372803312372198,
 0.867518394325971,
 0.9593548895680893,
 0.9017226623630116,
 0.5501496903546678,
 0.7457398999115231,
 0.4117873866332133,
 0.9793821813312402,
 0.4656491682341082,
 0.9881658194110153,
 0.30432118818065745,
 0.5753282550690745,
 0.4478519124357522,
 0.21946889413293802,
 0.14920276910823296,
 0.7278916423185626,
 0.9775290462529921,
 0.216713321183593,
 2.3354327679561103e-78,
 0.9478657766110571,
 0.3522344270428326,
 0.39047901551670006,
 0.46434067532656076,
 0.31789845905370756,
 0.9881635957019257,
 0.23557831814117214,
 0.8884265597817679,
 0.5204698790753718,
 0.7550783290113406,
 0.7305648561787131,
 0.5707270299456478,
 0.8596231565429879,
 0.3665181334302418,
 0.10586110305667251,
 0.31683434874235605,
 0.12490367190442593,
 0.6675722294734627,
 0.30306379232652664,
 0.8947177782768667,
 0.41628555791150207,
 0.5245751112484212,
 0.3298757969610609,
 0.2418213573270269,
 8.764792329201055e

In [84]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.49

In [85]:
bleu_min = min(bleu_scores)
bleu_min

0

In [86]:
bleu_max = max(bleu_scores)
bleu_max

1.0

In [87]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.4536791504382816

In [88]:
import sys
from nltk.translate.meteor_score import meteor_score

In [89]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [90]:
meteor_scores

[0.8132982719759579,
 0.6332571138211383,
 0.9601814516129031,
 0.125,
 0.9799587975278516,
 0.8803427031189833,
 0.8590521694214875,
 0.8259259259259261,
 0.8949194042032237,
 0.8160903250188963,
 0.9169724770642201,
 0.7823485876403108,
 0.9299633936261843,
 0.6763688843164152,
 0.8012820512820514,
 0.9109756097560976,
 0.34571428571428575,
 0.05050505050505051,
 0.8097560975609756,
 0.9240225015626085,
 0.6222271517302574,
 0.0,
 0.9283601014370245,
 0.5480555555555556,
 0.8187416978625769,
 0.5902439024390245,
 0.2808302808302808,
 0.999898229187869,
 0.19965277777777776,
 0.9355582967836257,
 0.8815701929474385,
 0.9679878048780488,
 0.7612635218021635,
 0.6225,
 0.9622367465504721,
 0.860948667966212,
 0.0423728813559322,
 0.3507042253521127,
 0.0,
 0.7134051845590308,
 0.09803921568627452,
 0.9875337577160495,
 0.35714285714285715,
 0.3457364341085272,
 0.125,
 0.03937007874015748,
 0.0,
 0.8470892137096774,
 0.04201680672268908,
 0.9208848012889366,
 0.19508057675996607,
 0.981

In [91]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.62

In [92]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.999898229187869

In [93]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [95]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.7633498391672242

In [ ]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
from bert_score import score

In [ ]:
!git clone https://github.com/Tiiiger/bert_score.git

Cloning into 'bert_score'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 993 (delta 219), reused 322 (delta 198), pack-reused 638
Receiving objects: 100% (993/993), 1.34 MiB | 10.33 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [ ]:
bert_scores_P = []
bert_scores_R = []
bert_scores_F1 = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  P, R, F1 = score(gen_spoiler_strip, actual_spoiler_strip, lang="en", verbose=True)
  bert_scores_P.append(P)
  bert_scores_R.append(R)
  bert_scores_F1.append(F1)

  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  # meteor_scores.append(meteor)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:13                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/bert_score/score.py:70 in score                          │
│                                                                                                  │
│    67 │   │   │   │     multiple references, the returned score of this candidate is             │
│    68 │   │   │   │     the *best* score among all references.                                   │
│    69 │   """                                                                                    │
│ ❱  70 │   assert len(cands) == len(refs), "Different number of candidates and references"        │
│    71 │                                                                                          │
│    72 │   assert (                                                                               │
│    73 │   │   lang is not None or model_type is not None                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: Different number of candidates and references

In [ ]:
# P, R, F1 = score(cands, refs, lang="en", verbose=True)

In [ ]:
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  print("\n")
  print(ref)
  print("\n")
  print(hypo)
  print("\n")



['tom', 'brady', "'", 's', 'definition', 'of', 'catching', 'up', 'involves', 'much', 'more', 'than', 'just', 'a', 'meal', '.', 'wes', 'welker', 'contacted', 'the', 'patriots', 'quarterback', 'friday', 'morning', 'on', 'a', 'miami', 'radio', 'station', '.', 'bra']


['how', 'about', 'that', 'morning', 'we', 'go', 'throw', '?']




['nasa', 'scientists', 'are', 'predicting', 'the', 'hole', 'in', 'the']


['2070']




['research', 'by', 'hiring', 'software', 'provider', 'cangrade', 'found', 'that', 'being', 'intellectually', 'stimulated']


['intellectual', 'stimulation']




['cal', 'newport', "'", 's', 'investigation', 'into', 'how', 'passionate', 'people', 'like', 'steve', 'jobs', 'really', 'got', 'started', 'as', 'well', 'as', 'what', 'scientists', 'say', 'predicts', 'happiness', 'and', 'fuels', 'great', 'accomplishment', 'newport', '.', 'ryan', 'holiday', 'author', 'of', 'ego', 'is', 'the', 'enemy', 'your', 'passion', 'may', 'be', 'the', 'very', 'enemy', '.', 'the', 'only', 'way', 

In [ ]:
bert_scores_P

In [ ]:
bert_scores_R

In [ ]:
bert_scores_F1